<a href="https://colab.research.google.com/github/Kobai/PokeType/blob/master/poketype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing all the necessary packages

In [0]:
import urllib.request
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

## Download all the images from the pokémon website

In [0]:
os.mkdir('./imgs')
os.mkdir('./eval')
os.mkdir('./predict')
os.mkdir('./predict/imgs')

for i in range(649):
  num = '%03d' % (i+1)
  urllib.request.urlretrieve('https://assets.pokemon.com/assets/cms2/img/pokedex/full/%s.png' % num, './imgs/%s.png' % str(i+1))

for i in range(650,722):
  num = '%03d' % i
  urllib.request.urlretrieve('https://assets.pokemon.com/assets/cms2/img/pokedex/full/%s.png' % num, './eval/%s.png' % str(i))

## Move all the images to type based folders


In [0]:
# We use a csv from kaggle and index the pokemon image's id to find the appropriate type
df = pd.read_csv('pokemon_stat.csv')
train_list = df.head(649)['type_1'].to_list()
eval_list = df.head(721).tail(721-649)['type_1'].to_list()
all_types = list(set(train_list))

for ptype in all_types:
  os.mkdir('./imgs/%s' % ptype)

for ptype in all_types:
  os.mkdir('./eval/%s' % ptype)

for img in os.listdir('./imgs'):
  if re.match('\d+.png', img):
    index = int(img.split('.')[0])
    os.rename('./imgs/%s' % img, './imgs/%s/%s' % (train_list[index-1], img))

for img in os.listdir('./eval'):
  if re.match('\d+.png', img):
    index = int(img.split('.')[0])
    os.rename('./eval/%s' % img, './eval/%s/%s' % (eval_list[index-1-649], img))

## Prepare the data to be loaded into the model

In [0]:
# Transformations to apply to the train and test sets
train_transform = transforms.Compose([
    transforms.Resize(80),
    transforms.CenterCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) 

test_transform = transforms.Compose([
    transforms.Resize(80),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create datasets from the image folders and apply transformations
train_data = datasets.ImageFolder('imgs', transform=train_transform)
test_data = datasets.ImageFolder('eval', transform=test_transform)

# Create dataloaders from the datasets
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

## View some the images to see what the transformations look like

In [0]:
classes = ['bug', 'dark', 'dragon', 'electric', 'fairy', 'fighting', 'fire', 'flying', 'ghost', 'grass', 'ground', 'ice', 'normal', 'poison', 'psychic', 'rock', 'steel', 'water']
dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.numpy() 

img = plt.figure(figsize=(25, 4))
for i in range(20):
  ax = img.add_subplot(2, 20/2, i+1, xticks=[], yticks=[])
  plt.imshow(np.transpose(images[i], (1, 2, 0)))
  ax.set_title(classes[labels[i]])

In [192]:
images.shape

(32, 3, 64, 64)

## Define a CNN model

In [0]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)

    self.pool = nn.MaxPool2d(2,2)

    self.dropout = nn.Dropout(0.25)

    self.fc1 = nn.Linear(64 * 8 * 8, 512)
    self.fc2 = nn.Linear(512, 18)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 64 * 8 * 8)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

model = Net()
print(model)
model.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

## Train the model

In [0]:
model.train()
running_loss = 0.0

for epoch in range(30):
  for i, batch in enumerate(trainloader, 0):
    data, target = batch
    data, target = data.cuda(), target.cuda()

    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 10 == 9:
      print(f'Epoch: {epoch+1} / Running Loss: {running_loss/10}')
      running_loss = 0.0

In [0]:
torch.save(model.state_dict(), 'poke_model.pt')

## Test the model

In [226]:
test_loss = 0.0
class_correct = list(0. for i in range(18))
class_total = list(0. for i in range(18))

with torch.no_grad():
  for data, target in testloader:
    data, target = data.cuda(), target.cuda()

    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)

    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(np.squeeze(correct_tensor.cpu().numpy()))

    for i in range(len(target.data)):
      label = target.data[i]
      class_correct[label] += correct[i].item()
      class_total[label] += 1

test_loss = test_loss/len(testloader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

Test Loss: 4.811930



In [227]:
print(f'Accuracy: {100. * np.sum(class_correct) / np.sum(class_total)}%')

Accuracy: 19.444444444444443%
